# AutoChef

## Prepare environment and process data

Data Source: https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews?resource=download

Data Source v2: https://app.roboflow.com/bens-workspace-3xdyh/fridge-detection-aymme/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true

In [3]:
import os
from ultralytics import YOLO
import torch
import cv2
import requests
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

## Take 2 - use YOLO for a simplified approach

In [11]:

print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

print(torch.__version__)
print(torch.version.cuda)


CUDA Available: True
Device Name: NVIDIA GeForce GTX 1660
2.3.1+cu118
11.8
Torch: 2.3.1+cu118
Torchvision: 0.18.1+cu118
CUDA available: True
tensor([0], device='cuda:0')


### Core functionality

In [4]:
# Load your trained model
model = YOLO('best_v2.pt')

# Load an image
img = cv2.imread('fridge_test_2.jpg')

# Run inference
results = model(img)

# Print detected objects
for result in results:
    boxes = result.boxes
    for box in boxes:
        class_idx = int(box.cls)
        class_name = model.names[class_idx]
        confidence = float(box.conf)
        print(f"Class: {class_name}, Confidence: {confidence:.2f}, Box: {box.xyxy}")

confidence_threshold = 0.7

for result in results:
    # Filter boxes by confidence
    high_conf_boxes = [box for box in result.boxes if float(box.conf) > confidence_threshold]
    if high_conf_boxes:
        # Optionally, update result.boxes to only include high confidence boxes
        result.boxes = high_conf_boxes
        result.show()

RuntimeError: Numpy is not available

In [4]:
# Get a list of detected classes
detected_classes = [model.names[int(box.cls)] for box in results[0].boxes if float(box.conf) > confidence_threshold]
print("Detected classes:", detected_classes)

Detected classes: ['tomato', 'apple', 'chicken', 'spinach', 'sugar', 'ground_beef', 'corn', 'green_beans', 'butter', 'sugar', 'ham']


In [15]:
# Get recipes that can be made with the detected ingredients
url = "https://api.spoonacular.com/recipes/findByIngredients"

params = {
    "ingredients": ",".join(detected_classes),
    "ignorePantry": True,
    "ranking": 2,
    "number": 5,
    'apiKey': os.getenv("SPOONACULAR_API_KEY")
}

#print("Requesting URL with params:", params)
response = requests.get(url, params=params)

response = requests.get(
    url,
    params=params
)

print(response.json())
data = response.json()

[{'id': 769775, 'title': 'Simple Spinach and Tomato Frittata', 'image': 'https://img.spoonacular.com/recipes/769775-312x231.jpg', 'imageType': 'jpg', 'usedIngredientCount': 2, 'missedIngredientCount': 2, 'missedIngredients': [{'id': 99037, 'amount': 1.0, 'unit': 'small', 'unitLong': 'small', 'unitShort': 'small', 'aisle': 'Pasta and Rice', 'name': 'personal skillet', 'original': 'small personal skillet', 'originalName': 'personal skillet', 'meta': [], 'image': 'https://img.spoonacular.com/ingredients_100x100/no.jpg'}, {'id': 1123, 'amount': 2.0, 'unit': '', 'unitLong': '', 'unitShort': '', 'aisle': 'Milk, Eggs, Other Dairy', 'name': 'eggs', 'original': '2-3 lightly beaten eggs', 'originalName': 'lightly beaten eggs', 'meta': ['lightly beaten'], 'image': 'https://img.spoonacular.com/ingredients_100x100/egg.png'}], 'usedIngredients': [{'id': 10311529, 'amount': 1.0, 'unit': 'serving', 'unitLong': 'serving', 'unitShort': 'serving', 'aisle': 'Produce', 'name': 'cherry tomatoes', 'original'

In [10]:
recommended_recipes = []
for recipes in data:
    print(f"Recipe ID: {recipes['id']}, Title: {recipes['title']}, Used Ingredients: {len(recipes['usedIngredients'])}, Missed Ingredients: {len(recipes['missedIngredients'])}")
    print("Used Ingredients:", [ingredient['name'] for ingredient in recipes['usedIngredients']])
    print("Missed Ingredients:", [ingredient['name'] for ingredient in recipes['missedIngredients']])
    print("-" * 40)
    recommended_recipes.append({
        "id": recipes['id'],
        "image": recipes['image'],
        "title": recipes['title'],
        "used_ingredients": [ingredient['name'] for ingredient in recipes['usedIngredients']],
        "missed_ingredients": [ingredient['name'] for ingredient in recipes['missedIngredients']]
    })

Recipe ID: 769775, Title: Simple Spinach and Tomato Frittata, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['cherry tomatoes', 'spinach leaves']
Missed Ingredients: ['personal skillet', 'eggs']
----------------------------------------
Recipe ID: 663748, Title: Traditional Apple Tart, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['granny smiths', 'butter']
Missed Ingredients: ['blackcurrant jam', 'oz short crust pastry - ready made if you want']
----------------------------------------
Recipe ID: 640089, Title: Corn on the Cob in Cilantro and Lime Butter, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['butter', 'corn on the cob']
Missed Ingredients: ['cilantro', 'lime']
----------------------------------------
Recipe ID: 645733, Title: Grilled Ham and Swiss Sandwich, Used Ingredients: 2, Missed Ingredients: 2
Used Ingredients: ['pats of butter', 'ham']
Missed Ingredients: ['swiss cheese', 'bread']
----------------------------------------
Re

In [1]:
from ultralytics import YOLO

model = YOLO('best_v2.pt')
model.save('best_v2_8.0.35.pt')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\bps78\Documents\GitHub\AutoDJ\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\bps78\Documents\GitHub\AutoDJ\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\bps78\Documents\GitHub\AutoDJ\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io